In [2]:
import pandas as pd
# from google.colab import files

import numpy as np
from scipy.spatial import distance_matrix
from gurobipy import *
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, Point
from numpy import random

In [5]:
data = pd.read_csv("광진구데이터프레임.csv")
data

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수,근처횡단보도수,사고건수
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0,221.0,11724.0,188.0,124.0,4.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0,46.0,2822.0,73.0,79.0,12.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0,47.0,2527.0,83.0,53.0,17.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0,42.0,2699.0,45.0,77.0,14.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0,44.0,1553.0,62.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
780,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0,77.0,55.0,2363.0,77.0,61.0,10.0
781,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0,68.0,49.0,2082.0,35.0,59.0,12.0
782,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0,48.0,212.0,11728.0,196.0,131.0,6.0
783,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0,9.0,23.0,1758.0,132.0,53.0,17.0


In [24]:
import collections
# 정원 수 100명 이상
# 근처수강생총합, 근처휴게음식점수, 사고건수 상위 50%로 나온 학원들을 추려 후보지로.
l1 = data[data['인원']>=100].index.to_list()
l2 = data[data['근처수강생총합']>=data['근처수강생총합'].quantile(0.5)].index.to_list()
l3 = data[data['근처휴게음식점수']>=data['근처휴게음식점수'].quantile(0.5)].index.to_list()
# l4 = data[data['사고건수']>=data['사고건수'].quantile(0.5)].index.to_list()
l4 = data[data['사고건수']>=0].index.to_list()

intersection = list(set(l1) & set(l2) & set(l3)&set(l4))

In [25]:
len(intersection)

37

In [26]:
data.loc[intersection, :]

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수,근처횡단보도수,사고건수
769,임박사의일본가자통역학원,서울특별시 광진구 구의강변로 83,37.538148,127.093503,278,139.0,139.0,160.0,9004.0,245.0,136.0,6.0
267,크레버스영재교육센터광진제1지점학원,서울특별시 광진구 광나루로 603,37.542015,127.095597,100,120.0,100.0,214.0,11565.0,186.0,98.0,2.0
144,광진일관정상어학원,서울특별시 광진구 아차산로 462,37.539496,127.093418,160,120.0,120.0,207.0,11356.0,228.0,137.0,6.0
272,광진심슨어학보습학원,서울특별시 광진구 광나루로 604,37.541573,127.094910,780,280.0,280.0,209.0,11247.0,186.0,91.0,2.0
406,웅진씽크빅학습센터광장학원,서울특별시 광진구 광나루로 607,37.541961,127.095718,155,124.0,124.0,215.0,11679.0,186.0,96.0,2.0
667,클라비스수학학원,서울특별시 광진구 구의로16길 89,37.541695,127.094384,420,103.0,103.0,210.0,11340.0,191.0,95.0,4.0
673,웅진프라임학원,서울특별시 광진구 광나루로 590,37.542576,127.093917,100,108.0,100.0,201.0,11309.0,174.0,81.0,4.0
422,최선생수학본영어학원,서울특별시 광진구 광나루로 602,37.541787,127.094700,250,112.0,112.0,209.0,11323.0,186.0,96.0,4.0
296,자양심슨어학학원,서울특별시 광진구 뚝섬로 552,37.534244,127.071276,900,296.0,296.0,109.0,4106.0,152.0,59.0,7.0
55,샤인영어학원,서울특별시 광진구 아차산로65길 14,37.541047,127.094089,106,136.0,106.0,203.0,11450.0,193.0,118.0,2.0
